In [1]:
from scipy.stats import qmc
import pandas as pd
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import shutil
import os
from sklearn.metrics import mean_squared_error
from math import sqrt
import subprocess
import warnings
import matplotlib.pyplot as plt
import geopandas as gpd
from matplotlib.colors import ListedColormap
import contextily as ctx
import matplotlib.colors as mcolors
warnings.filterwarnings("ignore")

### In this Notebook we make the all forest landcover inputs for heatsource scenario testing:
- Find grasst nodes by filtering shade values form a model run without topographic shade
- Make all nodes landcover equal to the forested buffer scenario
- if nodes were previously grassy, they are widened by %20 in the new forested model 

In [2]:
#reading in data with relative paths so that it can be used by others who clone repository from github
#os.chdir(os.getcwd())
script_dir = os.getcwd()


base_source_folder = os.path.abspath(os.path.join(script_dir, 'scenarios', 'scenario_inputs', 'baseline_vegetation', 'July_baseline'))

#reading in a model run without topographic shading, for use in determining whether a reach is forested or not
notopo_source_folder = os.path.abspath(os.path.join(script_dir, 'scenarios','scenario_inputs', 'notopo_models','july_notopo_extended'))

#lc data scenario with buffer, to be used for adding grass to previously forested areas
buffer_source_folder = os.path.abspath(os.path.join(script_dir, 'scenarios', 'scenario_inputs', 'vegetation_buffers', 'july_allforest_extended'))

#Temperature Observations folder
observed_temp_path = os.path.abspath(os.path.join(script_dir, 'observations', 'TemperatureDataObs'))
#Temperature observations
obs = pd.read_csv(os.path.join(observed_temp_path, 'HourlySummer23Master_riverkm.csv'))

In [3]:
#getting date times for modeled outputs
control_file = pd.read_csv(os.path.join(base_source_folder, 'HeatSource_Control.csv'))
cc_start = control_file.loc[7]['VALUE']
cc_start = pd.to_datetime(cc_start)
cc_end = control_file.loc[8]['VALUE']
cc_end = pd.to_datetime(cc_end)

#reading observed hourly temps for model range (for joining datetimes)
obs["datetime"] = pd.to_datetime(obs["datetime"])
obs_clip = obs[(obs["datetime"]>='07-01-2023') & (obs["datetime"]<'07-06-2023')]# + pd.DateOffset(1))]
#obs_clip = obs[(obs["datetime"]>=cc_start) & (obs["datetime"]<cc_end + pd.DateOffset(1))]

# Make observed temps compatible with heatsource outputs
obs_clip = obs_clip.dropna(axis=1)
obs_clip = obs_clip.set_index('datetime')
#obs_clip.columns = [float(col) for col in obs_clip.columns[2:]]
obs_clip.columns = [float(col) for col in obs_clip.columns]
#obs_clip.drop(19.2, axis=1, inplace=True)
obs_clip_mean = obs_clip.resample('D').mean()

In [4]:
notopo_shade_df = pd.read_csv(os.path.join(notopo_source_folder, 'outputs\Shade.csv'), skiprows=6,
         usecols=range(1, 187))
notopo_shade_df.columns = [float(col) for col in notopo_shade_df.columns]
notopo_shade_df = notopo_shade_df.mean()
notopo_shade_df = pd.DataFrame(notopo_shade_df)
notopo_shade_df = notopo_shade_df.reset_index()
#notopo_shade_df.index = obs_clip_mean.index
notopo_shade_df.rename(columns={'index': 'NODE_ID', 0 : 'July Shade'}, inplace=True)


In [5]:
## Scenario Type: All Grass Baseline
landcover = pd.read_csv(os.path.join(base_source_folder, 'inputs\WFK23_lcdata.csv'))
morphology = pd.read_csv(os.path.join(base_source_folder, 'inputs\WFK23_morphology.csv'))
landcover_notopo = pd.read_csv(os.path.join(notopo_source_folder, 'inputs\WFK23_lcdata.csv'))
landcover_buffer = pd.read_csv(os.path.join(buffer_source_folder, 'inputs\WFK23_lcdata.csv'))
changed_nodes = {}

#making a new folder for allgrass scenario
original_folder_path = base_source_folder
new_folder_path = os.path.abspath(os.path.join(script_dir, 'scenarios', 'scenario_inputs', 'sensitivity_analysis','july_allforest_widen40'))
shutil.copytree(original_folder_path, new_folder_path)

#adjust control file so it write output to correct folder
control_file = pd.read_csv(os.path.join(new_folder_path, 'HeatSource_Control.csv'))
control_file.loc[2, 'VALUE'] = str(os.path.join(new_folder_path, "inputs\\"))
control_file.loc[3, 'VALUE'] = str(os.path.join(new_folder_path, "outputs\\"))
control_file.to_csv(os.path.join(new_folder_path, 'HeatSource_Control.csv'), index = False)

landcover_km = landcover.copy()# Create a copy of landcover DataFrame
morphology_km = morphology.copy()# Create a copy of morphology DataFrame
reach_shade = notopo_shade_df#[(notopo_shade_df['NODE_ID'] <= km_start) & (notopo_shade_df['NODE_ID']>=km_end)]
##print(km_start)
    #changed_nodes[km_start] = []

#loop to remove shade at each node, narrows channel only where its currently grassy (where shade >.2 in baseline scenario)

for index, row in reach_shade.iterrows():
    #assigns bankbuffer shade to each node
    landcover_km.loc[index, landcover_km.columns[8:48]] = landcover_buffer.loc[index, landcover_buffer.columns[8:48]]
    landcover_km.to_csv(os.path.join(new_folder_path, 'inputs\WFK23_lcdata.csv'), index = False)

    # only widens currently grassy nodes
    if reach_shade.loc[index]['July Shade'] < .2:
        print('widen @: ', index)


        #if you want to only increase shade at previously grassy nodes, uncomment the below line
        # landcover_km.loc[index, landcover_km.columns[8:48]] = landcover_bankbuffer.loc[index, landcover_km.columns[8:48]]
        print(morphology_km.loc[index,morphology_km.columns[5]])
        #narrow currently forested reaches by 20% when changed to grass
        morphology_km.loc[index,morphology_km.columns[5]] = morphology_km.loc[index,morphology_km.columns[5]]*1.4
        print(morphology_km.loc[index,morphology_km.columns[5]])
        morphology_km.to_csv(os.path.join(new_folder_path, 'inputs\WFK23_morphology.csv'), index = False)            #morphology_km.to_csv(os.path.join(new_folder_path, 'inputs\WFK23_morphology.csv'), index = False)
            # changed_nodes[km_start].append(index)

os.chdir(new_folder_path)
subprocess.run(['hs', 'run', '-t'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

widen @:  2
5.389598543
7.545437960199999
widen @:  5
4.372751137
6.1218515918
widen @:  6
7.083336333
9.916670866199999
widen @:  8
9.071462879
12.7000480306
widen @:  9
5.0321873
7.04506222
widen @:  11
5.929937958
8.3019131412
widen @:  12
3.489613109
4.8854583526
widen @:  13
6.313660924
8.839125293599999
widen @:  14
8.288192233
11.6034691262
widen @:  15
7.815002025
10.941002834999999
widen @:  16
10.17731632
14.248242847999999
widen @:  17
9.432117926
13.204965096399999
widen @:  18
4.79286773
6.710014822
widen @:  25
15.36211315
21.50695841
widen @:  36
11.71617167
16.402640337999998
widen @:  37
13.03583371
18.250167194
widen @:  39
12.74136942
17.837917188
widen @:  40
13.64047508
19.096665112
widen @:  41
9.549882447
13.3698354258
widen @:  44
21.62387
30.273418
widen @:  45
20.3098155
28.433741699999995
widen @:  46
10.10703162
14.149844268
widen @:  48
6.631798273
9.2845175822
widen @:  50
7.2758047
10.18612658
widen @:  51
9.644345937
13.502084311800001
widen @:  52
10.72

CompletedProcess(args=['hs', 'run', '-t'], returncode=0)

In [6]:
base_tw_df = pd.read_csv(os.path.join(base_source_folder, 'outputs', 'Temp_H2O.csv'), skiprows=6,
         usecols=range(1, 187))
base_tw_df.columns = [float(col) for col in base_tw_df.columns]
base_tw_df.index = obs_clip.index
#mean july
mean_daily_base = base_tw_df.resample('D').mean()
mean_july_base = base_tw_df.mean()
mean_base = pd.DataFrame(mean_july_base)
mean_base = mean_base.reset_index()
mean_base.rename(columns={'index': 'NODE_ID', 0 : 'Mean'}, inplace=True)
mean_base['distance downstream'] = 18.3 - mean_base['NODE_ID']


allshade_tw_df = pd.read_csv(os.path.join(new_folder_path, 'outputs', 'Temp_H2O.csv'), skiprows=6,
         usecols=range(1, 187))
allshade_tw_df.columns = [float(col) for col in allshade_tw_df.columns]
allshade_tw_df.index = obs_clip.index
#mean july
mean_allshade = allshade_tw_df.resample('D').mean()
mean_allshade = mean_allshade.mean()
mean_allshade

mean_allshade = pd.DataFrame(mean_allshade)
mean_allshade = mean_allshade.reset_index()
mean_allshade.rename(columns={'index': 'NODE_ID', 0 : 'Mean'}, inplace=True)
mean_allshade['distance downstream'] = 18.3 - mean_allshade['NODE_ID']


ValueError: Length mismatch: Expected axis has 744 elements, new values have 120 elements

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(mean_base.index/10, mean_base['Mean'], label='July Base Model', color = 'black')
plt.plot(mean_allshade.index/10, mean_allshade['Mean'], label='July All Shade Model', color = 'blue')

#plt.plot(max_dailymean_scenario.index/10,max_dailymean_scenario['Max Daily Mean July Temp'],  label = 'Shade Scenario Model')


plt.ylabel('Maximum Average Daily Temp (c)')
plt.xlabel('Distance Downstream')
#plt.figure(facecolor='black')

    # Adjust layout
plt.tight_layout()
plt.grid(alpha = .2)
#plt.ylim(21, 25)
#plt.vlines(trib_dict_downstream, ymin = 20, ymax = 25, label = 'Tribs',color = 'Orange', linestyles = 'dashed')
plt.legend()
plt.title('July Mean Daily Temp WFK 2023 Heatsource Model')
plt.savefig('JulyDailyMean_withobstribQ.png')
plt.show()


NameError: name 'mean_base' is not defined

<Figure size 1000x700 with 0 Axes>